In [6]:
#https://keras.io/examples/vision/image_classification_efficientnet_fine_tuning/
from tensorflow.keras.applications import EfficientNetB0

In [3]:
import tensorflow as tf 
import pathlib 

def load_train_val_data(image_size=(384, 512), batch_size=16):
    base_dir = pathlib.Path('..')
    data_train = base_dir / 'data' / 'train'
    print('Loading data')
    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        '../data/train',
        validation_split=0.2,
        subset='training',
        label_mode='categorical',
        seed=1337,
        image_size=image_size,
        batch_size=batch_size,
    )

    val_ds = tf.keras.preprocessing.image_dataset_from_directory(
        data_train,
        validation_split=0.2,
        subset='validation',
        label_mode='categorical',
        seed=1337,
        image_size=image_size,
        batch_size=batch_size,
    )
    return train_ds, val_ds

In [4]:
train_ds, val_ds = load_train_val_data(image_size=(224,224))

Loading data
Found 1766 files belonging to 6 classes.
Using 1413 files for training.
Found 1766 files belonging to 6 classes.
Using 353 files for validation.


In [27]:
IMG_SIZE = 224
img_augmentation = tf.keras.models.Sequential(
    [
        layers.RandomRotation(factor=0.15),
        layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
        layers.RandomFlip(),
        layers.RandomContrast(factor=0.1),
    ],
    name="img_augmentation",
)

In [34]:
from tensorflow.keras import layers

def build_model(num_classes):
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    x = img_augmentation(inputs)
    model = EfficientNetB0(include_top=False, input_tensor=inputs, weights="imagenet")

    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.2
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(num_classes, activation="softmax", name="pred")(x)

    # Compile
    model = tf.keras.Model(inputs, outputs, name="EfficientNet")
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )
    return model

In [35]:
from matplotlib import pyplot as plt
def plot_hist(hist):
    plt.plot(hist.history["accuracy"])
    plt.plot(hist.history["val_accuracy"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()

In [36]:
#tf.keras.utils.plot_model(model, show_shapes=True)
model = build_model(num_classes=6)

In [37]:
epochs = 5
hist = model.fit(train_ds, epochs=epochs, validation_data=val_ds, verbose=2)

Epoch 1/5
89/89 - 27s - loss: 1.4343 - accuracy: 0.7183 - val_loss: 0.8002 - val_accuracy: 0.7394 - 27s/epoch - 303ms/step
Epoch 2/5
89/89 - 18s - loss: 0.7602 - accuracy: 0.8436 - val_loss: 0.7885 - val_accuracy: 0.7904 - 18s/epoch - 202ms/step
Epoch 3/5
89/89 - 18s - loss: 0.8869 - accuracy: 0.8351 - val_loss: 0.9491 - val_accuracy: 0.8159 - 18s/epoch - 200ms/step
Epoch 4/5
89/89 - 18s - loss: 0.6891 - accuracy: 0.8754 - val_loss: 1.2008 - val_accuracy: 0.7904 - 18s/epoch - 200ms/step
Epoch 5/5
89/89 - 18s - loss: 0.6103 - accuracy: 0.8825 - val_loss: 1.1810 - val_accuracy: 0.8300 - 18s/epoch - 200ms/step
